In [1]:
# 用來刪除測資和結果
import shutil
shutil.rmtree("data_graphform")
shutil.rmtree("result")

In [1]:
import random as rd
import os
import math
import random

def shuffle_G(G):
    target = [G[i][j] for i in range(len(G)) for j in range(len(G[0])) if G[i][j] != -1]
    origin_target = target.copy()
    while True:
        random.shuffle(target)
        is_clear = True
        for i in range(len(target)):
            if target[i] == origin_target[i] and target[i] != 0:
                is_clear = False
        if is_clear:
            break
    
    shuffled_G = [row.copy() for row in G]
        
    for i in range(len(shuffled_G)):
        for j in range(len(shuffled_G[0])):
            if shuffled_G[i][j] != -1:
                shuffled_G[i][j] = target[0]
                target = target[1:]
    
    return shuffled_G

def generate_robot_positions(size, num_robots):
    all_cells = [(i, j) for i in range(size[0]) for j in range(size[1])]
    robot_positions = random.sample(all_cells, num_robots)
    return robot_positions

def graph(t, size, n_sp, n_tar, num_robots, case, id):
    if num_robots > size[0] * size[1]:
        print(f"Warning: Number of robots ({num_robots}) exceeds grid size ({size[0]}x{size[1]}).")
        return False
    
    cell = [(i,j) for i in range(size[0]) for j in range(size[1])]
    G_start = [[-1 for j in range(size[1])] for i in range(size[0])]
    
    C_space = sorted(rd.sample(cell, n_sp))
    
    if C_space in case:
        return False
    
    case += [C_space]
    for i,j in C_space:
        G_start[i][j] = 0
    
    cargo_cells = [(i,j) for i,j in cell if G_start[i][j] == -1]
    choose = rd.sample(cargo_cells, n_tar)
    
    for index,(i,j) in enumerate(choose,1):
        G_start[i][j] = index
    
    robot_positions = generate_robot_positions(size, num_robots)
    
    G_end = shuffle_G(G_start)
    
    write(t, size, n_sp, n_tar, num_robots, G_start, G_end, robot_positions, id)
    return True

def write(t, size, n_sp, n_tar, num_robots, G_start, G_end, robot_positions, id):
    def write_to_file(filename, grid, robots):
        with open(filename, "w") as file:
            file.writelines(f"{t}\n")
            file.writelines(f"{len(robots)}\n")
            for robot in robots:
                file.writelines(f"{robot[0]},{robot[1]}\n")
            for i in range(size[0]):
                for j in range(size[1]):
                    if j != 0:
                        file.writelines(',')
                    file.writelines(f"{grid[i][j]:2}")
                file.writelines("\n")

    base_filename = f"{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_rob{len(robot_positions)}_{id}"
    
    # Write start file
    start_filename = f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\start\\graph_start_{base_filename}.txt"
    write_to_file(start_filename, G_start, robot_positions)
    
    # Write end file
    end_filename = f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\end\\graph_end_{base_filename}.txt"
    write_to_file(end_filename, G_end, robot_positions)


def special_case(size, n_sp, n_tar, num_robots, case, t):
    if num_robots > size[0] * size[1]:
        print(f"Warning: Number of robots ({num_robots}) exceeds grid size ({size[0]}x{size[1]}).")
        return False
    
    cells = sorted([(i,j) for i in range(size[0]) for j in range(size[1])],
                  key=lambda x: (x[0]+x[1],-x[0]*x[1]), reverse=True)
    
    G_start = [[-1 for j in range(size[1])] for i in range(size[0])]
    
    cargo_cells = cells[-1:-n_tar-1:-1]
    for index,(i,j) in enumerate(cargo_cells):
        G_start[i][j] = index+1
        
    space_cells = cells[:n_sp]
    for i,j in space_cells:
        G_start[i][j] = 0

    robot_positions = generate_robot_positions(size, num_robots)
    
    G_end = shuffle_G(G_start)
    write(t, size, n_sp, n_tar, num_robots, G_start, G_end, robot_positions, 0)
    case += [sorted([i for i in cells[:n_sp]])]
    return True


###-------- MAIN --------###
REPEAT = 3
size = [(4,4)]
robot_range = range(2, 10)  # 2到9台機器人
rd.seed(1)    # 固定seed，方便每次測資生成，亦可隨機

# 建構資料夾
if not os.path.exists("./data_graphform2"):
    os.mkdir("./data_graphform2")
if not os.path.exists("./result"):
    os.mkdir("./result")
    os.mkdir("./result/obstacle_noblock")
    os.mkdir("./result/obstacle_block")
    os.mkdir("./result/space_noblock")
    os.mkdir("./result/space_block")
    os.mkdir("./result/alg_multispace")

for s in size:  # 不同規模的倉儲系統
    if not os.path.exists(f"./data_graphform/{s[0]}x{s[1]}"):
        os.mkdir(f"./data_graphform/{s[0]}x{s[1]}")
        

    # 時間上限設置
    t=12
    
    
    tar_list = [4,5,6,7,8,9,10]
     
    
    for n_tar in tar_list:
        spacelist = list(range(0, 16 - n_tar))

        os.makedirs(f"./data_graphform/{s[0]}x{s[1]}/tar{n_tar}/start", exist_ok=True)
        os.makedirs(f"./data_graphform/{s[0]}x{s[1]}/tar{n_tar}/end", exist_ok=True)

        print(f"Target: {n_tar}, Spaces: {spacelist}")
        
        for n_sp in spacelist:
            for num_robots in robot_range:  # 遍歷不同的機器人數量
                case = []
                special_case(s, n_sp, n_tar, num_robots, case, t)
                
                for repeat in range(REPEAT):
                    check = graph(t, s, n_sp, n_tar, num_robots, case, repeat)
                    times = 0
                    while not check and times <= 20:
                        check = graph(t, s, n_sp, n_tar, num_robots, case, repeat)
                        times += 1
            
                with open(f"data_graphform/{s[0]}x{s[1]}/g_summary.txt","a") as file:
                    file.writelines(f"{n_tar} {n_sp} {num_robots} {REPEAT}\n")
                with open(f"data_graphform/summary.txt","a") as file:
                    file.writelines(f"{s[0]}x{s[1]} {n_tar} {n_sp} {num_robots} {REPEAT}\n")

        with open(f"data_graphform/{s[0]}x{s[1]}/g_summary.txt","a") as file:
            file.writelines(f"-------\n")
        with open(f"data_graphform/summary.txt","a") as file:
            file.writelines(f"-------\n")

Target: 4, Spaces: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Target: 5, Spaces: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Target: 6, Spaces: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Target: 7, Spaces: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Target: 8, Spaces: [0, 1, 2, 3, 4, 5, 6, 7]
Target: 9, Spaces: [0, 1, 2, 3, 4, 5, 6]
Target: 10, Spaces: [0, 1, 2, 3, 4, 5]
